# 프로젝트: ResNet Ablation Study

## 1) ResNet 기본 블록 구성하기

![resnetstruc](../image/deeper2ex_resnet.png)

In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from tensorflow.keras import layers

In [2]:
def normalize_and_resize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    # image = tf.image.resize(image, [32, 32])
    return tf.cast(image, tf.float32) / 255., label

In [3]:
def apply_normalize_on_dataset(ds, is_test=False, batch_size=16):
    ds = ds.map(
        normalize_and_resize_img, 
        num_parallel_calls=1
    )
    ds = ds.batch(batch_size)
    if not is_test:
        ds = ds.repeat()
        ds = ds.shuffle(200)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

In [4]:
import urllib3
urllib3.disable_warnings()

ds_train, ds_info = tfds.load(
    'cifar10',
    split='train[:1%]',
    as_supervised=True,
    shuffle_files=True,
    with_info=True,
)


In [5]:
print(ds_info.features)

FeaturesDict({
    'id': Text(shape=(), dtype=tf.string),
    'image': Image(shape=(32, 32, 3), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
})


In [6]:
# 데이터의 개수도 확인해 봅시다. 
print(tf.data.experimental.cardinality(ds_train))

tf.Tensor(500, shape=(), dtype=int64)


In [7]:
# ds_train = apply_normalize_on_dataset(ds_train, batch_size=100)

## 2) ResNet-34, ResNet-50 Complete Model

ResNet-34  
VGG와 같이 블록을 만드는 함수를 사용해서 직접 전체 모델을 만들어 봅시다.  
ResNet-34와 ResNet-50의 차이에 따라 달라지는 구성(configuration)을 함수에 전달해서  
같은 생성 함수 build_resnet()를 통해서 ResNet의 여러가지 버전들을 모두 만들어 낼 수 있도록 해야 합니다.

In [8]:
def build_resnet34(input_layer, filters=[64,64], num_cnn=2):
    x = keras.layers.MaxPool2D((3,3), strides=2)(input_layer)
    print('1:',x.shape)
    shortcut = x
    
    for i in range(num_cnn):
        if (i == 0):
            x = tf.keras.layers.Conv2D(filters[0], (3, 3), strides=(2, 2))(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.Activation('relu')(x)
            
            x = tf.keras.layers.Conv2D(filters[1], (3, 3), strides=(1, 1), padding='same')(x)
            x = tf.keras.layers.BatchNormalization()(x)
            
            x = tf.keras.layers.Add()([x, shortcut])
            x = tf.keras.layers.Activation('relu')(x)
            print(f'{i}:',x.shape)
            
        else:
            x = tf.keras.layers.Conv2D(filters[0], (3, 3), strides=(1, 1))(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.Activation('relu')(x)
            
            x = tf.keras.layers.Conv2D(filters[1], (3, 3), strides=(1, 1), padding='same')(x)
            x = tf.keras.layers.BatchNormalization()(x)
            
            x = tf.keras.layers.Add()([x, shortcut])
            x = tf.keras.layers.Activation('relu')(x)
            print(f'{i}:',x.shape)
            
            

In [9]:
# resnet블록 생성
def build_resnet50(input_layer, filters=[64,64,256], num_cnn=3):
    # 입력 레이어
    x = keras.layers.MaxPool2D((3,3), strides=2)(input_layer)
    shortcut = x 
    
    for i in range(num_cnn):
        if (i == 0):
            x = tf.keras.layers.Conv2D(filters[0], (1, 1), strides=(2, 2))(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.Activation('relu')(x)
            
            x = tf.keras.layers.Conv2D(filters[1], (3, 3), strides=(1, 1), padding='same')(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.Activation('relu')(x)
            x = tf.keras.layers.Conv2D(filters[2], (1, 1), strides=(1, 1))(x)
            x = tf.keras.layers.BatchNormalization()(x)
            
            shortcut = tf.keras.layers.Conv2D(filters[2], (1, 1), strides=(1, 1), padding='valid')(shortcut)            
            shortcut = tf.keras.layers.BatchNormalization()(shortcut)
 
            x = tf.keras.layers.Add()([x, shortcut])
            x = tf.keras.layers.Activation('relu')(x)
            
            shortcut = x
 
        else:
            x = tf.keras.layers.Conv2D(filters[0], (1, 1), strides=(1, 1), padding='valid')(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.Activation('relu')(x)
            
            x = tf.keras.layers.Conv2D(filters[1], (3, 3), strides=(1, 1), padding='same')(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.Activation('relu')(x)
 
            x = tf.keras.layers.Conv2D(filters[2], (1, 1), strides=(1, 1), padding='valid')(x)
            x = tf.keras.layers.BatchNormalization()(x)            
 
            x = tf.keras.layers.Add()([x, shortcut])   
            x = tf.keras.layers.Activation('relu')(x)  
 
            shortcut = x        
    
    return x
    
    

In [10]:
input_layer = tf.keras.layers.Input(shape=(64,64,3))

In [11]:
b3 = build_resnet34(input_layer)
b5 = build_resnet50(input_layer)

1: (None, 31, 31, 3)


ValueError: Operands could not be broadcast together with shapes (15, 15, 64) (31, 31, 3)

In [ ]:
## 차원이 다른데 어떻게 맞춰줘야하는지

In [ ]:
model = tf.keras.Model(inputs=input_layer, outputs=build_resnet) 

In [ ]:
model.summary()

In [ ]:
def ResNet34(input_shape=(32,32,3),num_classes=10):
    x = keras.layers.Input(input_shape)
    x = keras.layers.Conv2D(64, (3,3), padding='same', strides=2, activation='relu')(x)
    
    # block 1-3(with 2)
    x = build_resnet34(x, filters=[64,64], num_cnn=3)(x)
    x = build_resnet34(x, filters=[128,128], num_cnn=4)(x)
    x = build_resnet34(x, filters=[256,256], num_cnn=6)(x)
    x = build_resnet34(x, filters=[512,512], num_cnn=3)(x)
    
    
    
    
    x = keras.layers.AveragePooling2D()(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(num_classes, activation='softmax')(x)
    
    
    


In [ ]:
def resnet50(input_shape=(32,32,3),num_classes=10):
    input_tensor = tf.keras.Input(shape=input_shape, dtype='float32', name='input')
    x = keras.layers.Input(input_tensor)
    x = keras.layers.Conv2D(64, (7,7), padding='same', strides=2, activation='relu')(x)
    x = keras.layers.MaxPool2D((3,3), padding='same', strides=2)(x)
    
    x = build_resnet50(x, filters=[64,64,256], num_cnn=3)(x)
    x = build_resnet50(x, filters=[128,128,512], num_cnn=4)(x)
    x = build_resnet50(x, filters=[256,256,1024], num_cnn=6)(x)
    x = build_resnet50(x, filters=[512,512,2048], num_cnn=3)(x)
    
    x = keras.layers.AveragePooling2D()(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(num_classes, activation='softmax')(x)
    
    return x

        
    

    

In [ ]:
Resnet50 = resnet50()

In [ ]:
resnet50_ = tf.keras.Model(inputs=input_layer, outputs= resnet50_,num_classes=10)

In [ ]:
model = tf.keras.Model(inputs=input_layer, outputs=build_resnet) 